In [1]:
from parser_1 import read_recipe_from_url
import re
import sys
import os

folder_path = os.path.join(os.getcwd(), 'ourtypes')
sys.path.append(folder_path)

from step import Step, Steps
from ingredient import Ingredient

In [2]:
soup = read_recipe_from_url("https://www.allrecipes.com/recipe/255365/edible-cookie-dough/")

In [3]:
ingredients = soup.find("div", {"id":"mntl-structured-ingredients_1-0"})
for child in ingredients.find_all("li"):
    text = child.text
    text = text.replace("\n", "").strip()
    print(text)
    

1 cup all-purpose flour
¾ cup packed brown sugar
½ cup butter
1 teaspoon vanilla extract
½ teaspoon salt
2 tablespoons milk
½ cup milk chocolate chips
½ cup mini chocolate chips


In [5]:
ingredient_objects = {}
for child in ingredients.find_all("li"):
    text = child.text.replace("\n", "").split()[-1]
    ingredient_objects[text] = text

steps = soup.find("div", {"id":"recipe__steps-content_1-0"})
steps_collection = Steps()
time_pattern = r'for (.*? (minutes?|seconds?|hours?)\b)'

for child in steps.find_all("p"):
    step_text = child.text.replace("\n", "").strip()
    
    #find the time mentioned for in the step
    time_match = re.search(time_pattern, step_text)
    if time_match:
        time = time_match.group(1)
    else:
         time = None
    
    #find the temperature for that step
    temp = [" ".join([step_text.split()[i - 1], step_text.split()[i + 1]]) for i, word in enumerate(step_text.split()) if word == 'degrees']
    #identify ingredients present in this step
    step_ingredients = [ingredient_objects[ingr] for ingr in ingredient_objects if ingr in step_text]

    #create Step object
    step_obj = Step(step_ingredients, tools=[], methods=[], description=step_text, time=time, temp=temp)

    #add Step object to Steps collection
    steps_collection.add_step(step_obj)

print(steps_collection)

Step 1: Step(Ingredients: ['flour'], Tools: [], Methods: [], Time: 1 minute, Temp: [], Description: 'To heat-treat your flour so it is safe to use: Place flour in a microwave-safe dish and cook for 1 minute and 15 seconds, stirring it every 15 seconds. Set aside.')
Step 2: Step(Ingredients: ['flour', 'sugar', 'butter', 'extract', 'salt', 'milk', 'chips'], Tools: [], Methods: [], Time: None, Temp: [], Description: 'Beat sugar and butter with an electric mixer in a large bowl until creamy. Beat in vanilla extract and salt. Add heat-treated flour; mix until a crumbly dough forms. Stir in milk until dough is just combined; fold in milk chocolate chips and mini chocolate chips.')
Step 3: Step(Ingredients: ['flour'], Tools: [], Methods: [], Time: 5 to 6 minutes, Temp: ['350 F'], Description: 'There is a potential risk of foodborne illness from the consumption of raw flour. Follow Step 1 to heat-treat your flour so it is safe to use. If you do not have a microwave, spread flour out on a bakin